In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the dataset
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon*sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image


def train(model, device, train_loader, optimizer, epoch, epsilon):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data.requires_grad = True

        output = model(data)
        loss = F.nll_loss(output, target)
        optimizer.zero_grad()
        loss.backward()

        data_grad = data.grad.data
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        output = model(perturbed_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, 11):  # 10 epochs
    train(model, device, train_loader, optimizer, epoch, epsilon=0.25)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.329581
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.260731
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.800413
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.479351
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.046727
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.204394
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.095953
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.992430
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.772094
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.640707

Test set: Average loss: 0.1723, Accuracy: 9503/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.916796
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.745241
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.837983
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.665966
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.644782
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.638676
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.875869
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.660361
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.368506
T

In [ ]:
def test_on_adversarial_examples(model, device, test_loader, epsilon):
    model.eval()
    adversarial_loss = 0
    correct = 0
    total = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data.requires_grad = True

        # Forward pass
        output = model(data)
        loss = F.nll_loss(output, target)
        model.zero_grad()
        loss.backward()

        # Create adversarial examples
        data_grad = data.grad.data
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed images
        output = model(perturbed_data)
        final_loss = F.nll_loss(output, target)
        adversarial_loss += final_loss.item()

        # Check accuracy
        pred = output.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)

    avg_loss = adversarial_loss / total
    accuracy = 100. * correct / total
    print(f'\nTest on Adversarial Examples: Average loss: {avg_loss:.4f}, Accuracy: {correct}/{total} ({accuracy:.0f}%)\n')

In [ ]:
epsilon = 0.25  # The same epsilon used during training or a different one to test robustness
test_on_adversarial_examples(model, device, test_loader, epsilon)


Test on Adversarial Examples: Average loss: 0.0034, Accuracy: 9317/10000 (93%)



In [ ]:
test(model, device, test_loader)


Test set: Average loss: 0.0419, Accuracy: 9876/10000 (99%)

